In [1]:
import torch
from d2l import torch as d2l

In [2]:
def corr2d_multi_in(X, K):
    # Iterate through the 0th dimension (channel) of K first, then add them up
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of K, and each time, perform
    # cross-correlation operations with input X. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # Matrix multiplication in the fully connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

#### 1. Assume that we have two convolution kernels of size $k_1$ and $k_2$, respectively (with no nonlinearity in between).
1. Prove that the result of the operation can be expressed by a single convolution.
1. What is the dimensionality of the equivalent single convolution?
1. Is the converse true, i.e., can you always decompose a convolution into two smaller ones?

(1)

Convolution follows the associative law:

(X * K_1) * K_2 = X * (K_1 * K_2)

How to prove it ? I can only prove for the convolution operator for two function: 
https://math.stackexchange.com/questions/2170534/proof-of-associativity-of-convolution

(2)

The size of the new kernel is $(k_1+k_2-1, k_1+k_2-1)$

(3)

We cannot always decompose a convolution into two smaller ones.

If the rank of the kernel matrix is 1, it is separable.

https://blogs.mathworks.com/steve/2006/11/28/separable-convolution-part-2/


### 2. Assume an input of shape $c_\textrm{i}\times h\times w$ and a convolution kernel of shape $c_\textrm{o}\times c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$, padding of $(p_\textrm{h}, p_\textrm{w})$, and stride of $(s_\textrm{h}, s_\textrm{w})$.
1. What is the computational cost (multiplications and additions) for the forward propagation?
1. What is the memory footprint?
1. What is the memory footprint for the backward computation?
1. What is the computational cost for the backpropagation?

(1)

For each element in the output, there are $c_i \times k_h  \times k_w$ times multiplications and $c_o \times c_i \times k_h  \times k_w - 1$ times additions.

There are  $c_o \times [(h - k_h + p_h + s_h) / s_h] \times [(w - k_w + p_w + s_w) / s_w]$ elements in the output. So the computational cost is about:

$c_o * c_i \times k_h  \times k_w \times [(h - k_h + p_h + s_h) / s_h] \times [(w - k_w + p_w + s_w) / s_w]$ multiplications and additions

(2)

Memory Footprint = (Input Data Size + Kernel Parameters Size + Output Data Size + Activation Maps Size) × Data Type Size

(3)

Memory Footprint = (Gradients Size+Intermediate Values Size+Activation Maps Size+Workspace Size)×Data Type Size

(4)

Backpropagation involves computing gradients with respect to both the layer’s input and its parameters (weights and biases).

#### 3. By what factor does the number of calculations increase if we double both the number of input channels $c_\textrm{i}$ and the number of output channels $c_\textrm{o}$? What happens if we double the padding?

The number of calculations will be 4 times if we double both number of input and output channels.

When we double the padding, the additional calculation will be 

$$ c_o * c_i \times k_h  \times k_w / s_h / s_w *[(h - k_h + p_h + s_h) * p_w + (w - k_w + p_w + s_w) * p_h] $$

It is much smaller than the original computational cost.

#### 4. Are the variables `Y1` and `Y2` in the final example of this section exactly the same? Why?

In [6]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
float(torch.abs(Y1 - Y2).sum()) < 1e-6

True

In [7]:
(Y1 == Y2).all()

tensor(True)

They may be not exactly the same. Because in practice, slight differences may arise due to different calculation order and numerical limitations.

#### 5. Express convolutions as a matrix multiplication, even when the convolution window is not $1 \times 1$.

Refered to 7.2 Q4

#### 6. Your task is to implement fast convolutions with a $k \times k$ kernel. One of the algorithm candidates is to scan horizontally across the source, reading a $k$-wide strip and computing the $1$-wide output strip one value at a time. The alternative is to read a $k + \Delta$ wide strip and compute a $\Delta$-wide  output strip. Why is the latter preferable? Is there a limit to how large you should choose $\Delta$?

The wider strip allows for more opportunities for parallelism since you can perform computations for multiple output values simultaneously. This can be beneficial for optimizing computations on modern hardware, like GPUs or parallel CPU architectures.

The choice of $\triangle$ depends on factors such as memory constraints, cache size, and hardware architecture. 

#### 7. Assume that we have a $c \times c$ matrix. 
1. How much faster is it to multiply with a block-diagonal matrix if the matrix is broken up into $b$ blocks?
1. What is the downside of having $b$ blocks? How could you fix it, at least partly?

(1) 

Full Matrix Multiplication Complexity: For a c×c matrix multiplication, the complexity is $O(c^3)$. When the matrix is broken up into b blocks, the complexity becomes O((c/b)^3).

(2) 

Dividing a matrix into more blocks requires storing more intermediate results and indices, which can increase memory overhead. Instead of increasing the number of blocks ((b)), you can increase the size of each block. This can help reduce memory overhead and improve parallel efficiency. 